In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [3]:
import pymysql
import json
import warnings
import DB
warnings.filterwarnings(action='ignore')

In [4]:
def team_player_concat(team, player, team_rate):
  player_rate = 1 - team_rate
  output_data = pd.DataFrame()
  output_data['gmkey'] = team['gmkey']
  output_data['ast'] = team_rate * team['ast'] + player_rate * player['ast']
  # 2점
  output_data['twop'] = team_rate * team['twop'] + player_rate * player['twop']
  # 3점
  output_data['threep'] = team_rate * team['threep'] + player_rate * player['threep']
  # 자유투
  output_data['freep'] = team_rate * team['freep'] + player_rate * player['freep']
  # 공격 리바
  output_data['offr'] = team_rate * team['offr'] + player_rate * player['offr']
  # 속공
  # output_data['tfb'] = team['tfb'] / team['fgt']
  # 벤치득점
  # output_data['bench'] = team['benchScoreCn'] / team['score']
  # 속공득점
  # output_data['tfbp'] = team['fbScoreCn'] / team['score']
  # 페인트존 득점
  output_data['ppp'] = team_rate * team['ppp'] + player_rate * player['ppp']
  # 덩크
  output_data['dk'] = team_rate * team['dk'] + player_rate * player['dk']
  # 최다 연속 득점 비율
  # output_data['maxContiScore'] = team['maxContiScoreCn'] / team['score']
  # 최다 리드 점수차
  # output_data['maxLeadScore'] = (team['score'] - player['score']) / team['maxLeadScoreCn']
  # 득점 우위 시간
  # output_data['scoreHighTime'] = team['scoreHighTimeCn'] / 2400
  # 세컨찬스 득점
  # output_data['secChanceScore'] = team['secChanceScoreCn'] / team['score']
  # 턴오버
  # output_data['to'] = (team['to'] + team['tto']) / (team['to'] + team['tto'] + player['to'] + player['tto'])
  # output_data['to'] = (team['to'] + team['tto']) / (team['to'] + team['tto'] + player['to'] + player['tto'])

  output_data['defr'] = team_rate * team['defr'] + player_rate * player['defr']
  output_data['stl'] = team_rate * team['stl'] + player_rate * player['stl']
  output_data['bs'] = team_rate * team['bs'] + player_rate * player['bs']
  output_data['foul'] = team_rate * team['foul'] + player_rate * player['foul']
  output_data['tf'] = team_rate * team['tf'] + player_rate * player['tf']
  output_data['gd'] = team_rate * team['gd'] + player_rate * player['gd']

  output_data['score'] = team['score']
  
  # output_data['teamR'] = team['teamR'] / (team['teamR'] + player['teamR'])
  # result.append(output_data, ignore_index=True)
  # print(output_data)
  # result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return output_data

In [189]:
def calculate_data(h_data, a_data):
  output_data = pd.DataFrame()
  output_data['gmkey'] = h_data['gmkey']
  output_data['ast'] = h_data['ast'] / h_data['fgt']
  # 2점
  output_data['twop'] = h_data['fg'] / h_data['fgA']
  # 3점
  output_data['threep'] = h_data['threep'] / h_data['threepA']
  # 자유투
  output_data['freep'] = h_data['ft'] / h_data['ftA']
  # 공격 리바
  output_data['offr'] = h_data['offr'] / (h_data['offr'] + a_data['defr'])
  # 속공
  # output_data['tfb'] = h_data['tfb'] / h_data['fgt']
  # 벤치득점
  # output_data['bench'] = h_data['benchScoreCn'] / h_data['score']
  # 속공득점
  # output_data['tfbp'] = h_data['fbScoreCn'] / h_data['score']
  # 페인트존 득점
  output_data['ppp'] = h_data['pp'] / h_data['ppA']
  # 덩크
  output_data['dk'] = h_data['dk'] / h_data['dkA']
  # 최다 연속 득점 비율
  # output_data['maxContiScore'] = h_data['maxContiScoreCn'] / h_data['score']
  # 최다 리드 점수차
  # output_data['maxLeadScore'] = (h_data['score'] - a_data['score']) / h_data['maxLeadScoreCn']
  # 득점 우위 시간
  # output_data['scoreHighTime'] = h_data['scoreHighTimeCn'] / 2400
  # 세컨찬스 득점
  # output_data['secChanceScore'] = h_data['secChanceScoreCn'] / h_data['score']
  # 턴오버
  # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])
  # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])

  output_data['defr'] = h_data['defr'] / (a_data['offr'] + h_data['defr'])
  output_data['stl'] = h_data['stl'] / (h_data['stl'] + a_data['stl'])
  output_data['bs'] = h_data['bs'] / a_data['fgtA']
  output_data['foul'] = h_data['foul'] / (h_data['foul'] + a_data['foul'])
  output_data['tf'] = h_data['tf'] / (h_data['tf'] + a_data['tf'])
  output_data['gd'] = h_data['gd'] / (h_data['gd'] + a_data['gd'])

  output_data['score'] = h_data['score']
  
  # output_data['teamR'] = h_data['teamR'] / (h_data['teamR'] + a_data['teamR'])
  # result.append(output_data, ignore_index=True)
  # print(output_data)
  # result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return output_data


In [ ]:
# def get_keyplayer_data_inmemory():
player_record = pd.read_csv('player_record.csv', delimiter=';')
db = DB.MYDB()
db.cursor.execute("select * from game_meta where gameDate >= '20101015'")
meta_data = db.cursor.fetchall()
for idx, meta in enumerate(meta_data):
    gmkey = meta['gmkey']
    
    print(player_record[player_record['gmkey'] < gmkey])
    # print(idx, meta)
    break
db.db.close()
# get_keyplayer_data_inmemory()

In [ ]:
def naive_model():
    db = DB.MYDB()
    db.cursor.execute("select * from team_record where home_away = 'H' and gmkey >= 'S17' order by gmkey")
    team_hdata = pd.DataFrame(db.cursor.fetchall())
    team_hdata = team_hdata.drop(team_hdata[team_hdata['gmkey'].isin(['S37G13N10', 'S37G13N2', 'S39G13N2', 'S39G13N5'])].index)
    team_hdata = team_hdata.reset_index(drop=True)
    db.cursor.execute("select * from team_record where home_away = 'A' and gmkey >= 'S17' order by gmkey")
    team_adata = pd.DataFrame(db.cursor.fetchall())
    team_adata = team_adata.drop(team_adata[team_adata['gmkey'].isin(['S37G13N10', 'S37G13N2', 'S39G13N2', 'S39G13N5'])].index)
    team_adata = team_adata.reset_index(drop=True)
    db.db.close()

    player_hdata = pd.read_csv('h_data.csv', index_col=0)
    player_adata = pd.read_csv('a_data.csv', index_col=0)

    player_hdata.sort_values(by='gmkey', inplace=True)
    player_adata.sort_values(by='gmkey', inplace=True)
    team_hdata.sort_values(by='gmkey', inplace=True)
    team_adata.sort_values(by='gmkey', inplace=True)

    home_result = team_player_concat(calculate_data(team_hdata,team_adata), calculate_data(player_hdata, player_adata), 0)
    away_result = team_player_concat(calculate_data(team_adata,team_hdata), calculate_data(player_adata, player_hdata), 0)
    data = pd.concat([home_result, away_result], ignore_index=True)
    data.fillna(0, inplace=True)
    data.drop(columns='gmkey', inplace=True)

    return data


In [319]:
#최근 5경기 기록으로 모델 생성
def recent_5_model(save=False):
    team_record = pd.read_csv('team_recent_avg_record.csv', index_col=0)
    player_record = pd.read_csv('player_recent_avg_record.csv', index_col=0)

    player_record = player_record.drop(index = [541, 3073, 3075, 3697, 3699, 6595, 7202, 7229])

    team_record.sort_values('gmkey', inplace=True)
    team_record.reset_index(inplace=True)
    player_record.sort_values('gmkey', inplace=True)
    player_record.reset_index(inplace=True)

    h_team_data = team_record[team_record.index % 2 == 0].reset_index()
    a_team_data = team_record[team_record.index % 2 != 0].reset_index()
    h_player_data = team_record[team_record.index % 2 == 0].reset_index()
    a_player_data = team_record[team_record.index % 2 != 0].reset_index()

    team_data = pd.concat([calculate_data(h_team_data, a_team_data), calculate_data(a_team_data, h_team_data)], ignore_index=True)
    player_data = pd.concat([calculate_data(h_player_data, a_player_data), calculate_data(a_player_data, h_player_data)], ignore_index=True)
    if save:
        team_data.to_csv('team_calculate_date.csv')
        player_data.to_csv('player_calculate_date.csv')

    data = team_player_concat(team_data,player_data, 0.5)
    data.drop(columns='gmkey', inplace=True)
    data.fillna(0, inplace=True)

    return data


In [323]:
#선형 회귀
data = recent_5_model()
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
mean_squared_error(y_predict, y_test, squared = False)
model.score(X_test, y_test)



0.006739551445622882

In [332]:
#Decision Tree를 통한 중요한 feature 파악
features = X.columns.tolist()

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# Extracting the importances by sklearn 
importances_sk = tree.feature_importances_

# Creating a dataframe with the feature importance by sklearn
feature_importance_sk = {}
for i, feature in enumerate(features):
    feature_importance_sk[feature] = round(importances_sk[i],5)
feature_importance_sk

{'ast': 0.07883,
 'twop': 0.08972,
 'threep': 0.07672,
 'freep': 0.0915,
 'offr': 0.08256,
 'ppp': 0.08034,
 'dk': 0.04158,
 'defr': 0.08288,
 'stl': 0.07612,
 'bs': 0.0902,
 'foul': 0.11199,
 'tf': 0.02462,
 'gd': 0.07294}